In [43]:
# Task 1.1
from datetime import datetime, timedelta
import csv

def readFile(filename = "TOKEN.txt"):
    # Initialisation
    data = []
    file = open(filename, 'r') # Read mode
    reader = csv.reader(file)
    
    # Read from file
    for record in reader:
        newRecord = record
        newRecord[0] = str(newRecord[0]) # User's randomised ID
        newRecord[1] = str(newRecord[1]) # Randomised ID of user in proximity
        newRecord[2] = int(newRecord[2]) # RSSI
        newRecord[3] = datetime.strptime(newRecord[3], "%Y-%m-%d %H:%M:%S") # timestamp of encounter
        data.append(newRecord)
    
    file.close()
    return data

def generateTracingData(data):
    users = {}
    for user, other, rssi, date in data:
        # Set user data
        if users.get(user) == None:
            users[user] = {
                "closeContact": {},  # People in close contact with and the start timestamp
                "prolongedExposure" : {}, # People with prolonged exposure and end timestamp
                "currTimings": {}
            }
        
        
        userStartDate = users[user]["closeContact"].get(other) # Time of first contact
        
        # Close contact
        if rssi >= -30 and (userStartDate == None or userStartDate > date):
            users[user]["closeContact"][other] = date
            users[user]["currTimings"][other] = [date]
        # Continuous exposure
        elif rssi >= -30 and userStartDate != None and date - users[user]["currTimings"][other][-1] == timedelta(minutes=5):
            users[user]["currTimings"][other].append(date)
        # Reset if too long
        elif rssi >= -30 and userStartDate != None and (date - users[user]["currTimings"][other][-1] > timedelta(minutes=5)):
            users[user]["closeContact"][other] = date
            users[user]["currTimings"][other] = [date]
            
        # Prolonged Exposure
        if rssi >= -30 and userStartDate != None and date - userStartDate >= timedelta(minutes=15) and len(users[user]["currTimings"][other]) >= 4:
            users[user]["prolongedExposure"][other] = date
        
    return users

def task1_1():
    data = readFile()
    tracingData = generateTracingData(data)
    # Displaying people need to contact trace
    for user in tracingData:
        print("From", user)
        for prolongedExposure, date in tracingData[user]["prolongedExposure"].items():
            startDate = tracingData[user]["closeContact"][prolongedExposure]
            print("", prolongedExposure, startDate, date)
            #print(tracingData[user]["currTimings"][prolongedExposure])
    
task1_1()

From 57345286
 92452814 2020-07-29 08:30:00 2020-07-29 08:55:00
 14394568 2020-07-29 12:50:00 2020-07-29 13:15:00
 27524883 2020-07-30 09:00:00 2020-07-30 09:25:00


In [44]:
# Task 1.2

data = readFile()
tracingData = generateTracingData(data)
def is_close_contact(rid1, rid2):
    inCloseContact1 =  (tracingData.get(rid1) != None and 
                        rid2 in tracingData[rid1]["prolongedExposure"].keys())
    inCloseContact2 =  (tracingData.get(rid2) != None and
                        rid1 in tracingData[rid2]["prolongedExposure"].keys())
    
    if not (inCloseContact1 or inCloseContact2):
        return False, None, None, None
    else:
        if inCloseContact1:
            key1 = rid1
            key2 = rid2
        elif inCloseContact2:
            key1 = rid2
            key2 = rid1
        startDate = tracingData[key1]["closeContact"][key2]
        endDate = tracingData[key1]["prolongedExposure"][key2]
        
        return True, startDate, endDate, endDate - startDate

is_close_contact("57345286", "75348257")

(False, None, None, None)

In [45]:
is_close_contact("57345286", "45174591")

(False, None, None, None)

In [46]:
is_close_contact("57345286", "02548147")

(False, None, None, None)